# Fantasy Football Week One
## Data Collection

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

In [2]:
%%capture

from tqdm import tqdm_notebook as tqdm
from tqdm import tnrange
tqdm().pandas()

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Setting DataFrames

In [5]:
passing_df = pd.DataFrame(columns = ['PLAYER', 'POSITION', 'TEAM', 'GP', 'ATT', 'CMP', 'PCT', 'YDS', 'YDSpG', 'LNG',
                                    'TD', 'INT', 'SCK', 'YDSL', 'RATE'])

In [7]:
rushing_df = pd.DataFrame(columns = ['PLAYER', 'POSITION', 'TEAM', 'GP', 'ATT', 'YDS', 'YDSpG', 'AVG', 'TD', 'LNG'])

In [8]:
receiving_df = pd.DataFrame(columns = ['PLAYER', 'POSITION', 'TEAM', 'GP', 'REC', 'YDS', 'YDSpG', 'AVG', 'LNG', 'TD'])

In [69]:
kicking_df = pd.DataFrame(columns = ['PLAYER', 'POSITION', 'TEAM', 'GP', 'FGM_A', 'FG%', 'LNG', '_1_19', '_20_29', 
                                     '_30_39', '_40_49', 'OVER50', 'XPM_A', 'XP%', 'PTS'])

In [9]:
defense_df = pd.DataFrame(columns = ['PLAYER', 'POSITION', 'TEAM', 'GP', 'STK', 'ATK', 'CMB', 'INT', 'YDS', 'LNG', 
                                     'TD', 'FUM', 'FREC', 'FTD', 'SCK', 'PDEF', 'SFTY'])

### Name, Position & Team Functions

In [113]:
def get_name(soup):
    container = soup.find('tbody')
    names = []
    for i in range(len(container.findAll('a', class_ = ''))):
        if i % 2 != 0:
            name = container.findAll('a', class_ = '')[i].get_text()
            names.append(name)
        else:
            continue
    return names

In [205]:
def get_name_pos_team_missing_team(soup):
    container = soup.find('tbody')
    names = []
    positions = []
    teams = []
    for i in range(len(container.findAll('span', class_ = 'CellPlayerName--long'))):
        name = container.findAll('span', class_ = 'CellPlayerName--long')[i].get_text()
        name = name.lstrip('\n')
        name = name.partition('\n')[0]
        names.append(name)
        
        position = container.findAll('span', class_ = 'CellPlayerName--long')[i].get_text()
        position = position.lstrip('\n')
        position = position.split('\n')
        position = position[2]
        position = position.replace(' ', '')
        positions.append(position)
        
        team = container.findAll('span', class_ = 'CellPlayerName--long')[i].get_text()
        team = team.lstrip('\n')
        team = team.split('\n')
        team = team[5]
        team = team.replace(' ', '')
        teams.append(team)
        
    return names, positions, teams

In [33]:
def get_position(soup):
    container = soup.find('tbody')
    positions = []
    for i in range(len(container.findAll('span', class_ = 'CellPlayerName-position'))):
        if i % 2 != 0:
            position = container.findAll('span', class_ = 'CellPlayerName-position')[i].get_text()
            position = position.replace('\n', '')
            position = position.replace(' ', '')
            positions.append(position)
        else:
            continue
    return positions

In [34]:
def get_team(soup):
    container = soup.find('tbody')
    teams = []
    for i in range(len(container.findAll('span', class_ = 'CellPlayerName-team'))):
        if i % 2 != 0:
            team = container.findAll('span', class_ = 'CellPlayerName-team')[i].get_text()
            team = team.replace('\n', '')
            team = team.replace(' ', '')
            teams.append(team)
        else:
            continue
    return teams

### 12 Stats

In [10]:
def get_12first(soup):
    container = soup.find('tbody')
    first_col = []
    count = 0
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            first_col.append(val)
        
        count += 12    
    return first_col

In [11]:
def get_12second(soup):
    container = soup.find('tbody')
    second_col = []
    count = 1
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            second_col.append(val)
        
        count += 12    
    return second_col

In [12]:
def get_12third(soup):
    container = soup.find('tbody')
    third_col = []
    count = 2
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            third_col.append(val)
        
        count += 12    
    return third_col

In [71]:
def get_12fourth(soup):
    container = soup.find('tbody')
    fourth_col = []
    count = 3
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            fourth_col.append(val)
        
        count += 12    
    return fourth_col

In [72]:
def get_12fifth(soup):
    container = soup.find('tbody')
    fifth_col = []
    count = 4
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            fifth_col.append(val)
        
        count += 12    
    return fifth_col

In [73]:
def get_12sixth(soup):
    container = soup.find('tbody')
    sixth_col = []
    count = 5
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            sixth_col.append(val)
        
        count += 12    
    return sixth_col

In [74]:
def get_12seventh(soup):
    container = soup.find('tbody')
    seventh_col = []
    count = 6
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            seventh_col.append(val)
        
        count += 12    
    return seventh_col

In [75]:
def get_12eighth(soup):
    container = soup.find('tbody')
    eighth_col = []
    count = 7
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            eighth_col.append(val)
        
        count += 12    
    return eighth_col

In [76]:
def get_12ninth(soup):
    container = soup.find('tbody')
    ninth_col = []
    count = 8
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            ninth_col.append(val)
        
        count += 12    
    return ninth_col

In [77]:
def get_12tenth(soup):
    container = soup.find('tbody')
    tenth_col = []
    count = 9
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            tenth_col.append(val)
        
        count += 12    
    return tenth_col

In [78]:
def get_12eleventh(soup):
    container = soup.find('tbody')
    eleventh_col = []
    count = 10
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            eleventh_col.append(val)
        
        count += 12    
    return eleventh_col

In [79]:
def get_12twelfth(soup):
    container = soup.find('tbody')
    twelfth_col = []
    count = 11
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            twelfth_col.append(val)
        
        count += 12    
    return twelfth_col

### 7 Stats

In [26]:
def get_7first(soup):
    container = soup.find('tbody')
    first_col = []
    count = 0
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            first_col.append(val)
        
        count += 7    
    return first_col

In [23]:
def get_7second(soup):
    container = soup.find('tbody')
    second_col = []
    count = 1
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            second_col.append(val)
        
        count += 7    
    return second_col

In [25]:
def get_7third(soup):
    container = soup.find('tbody')
    third_col = []
    count = 2
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            third_col.append(val)
        
        count += 7    
    return third_col

In [24]:
def get_7fourth(soup):
    container = soup.find('tbody')
    fourth_col = []
    count = 3
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            fourth_col.append(val)
        
        count += 7    
    return fourth_col

In [27]:
def get_7fifth(soup):
    container = soup.find('tbody')
    fifth_col = []
    count = 4
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            fifth_col.append(val)
        
        count += 7    
    return fifth_col

In [28]:
def get_7sixth(soup):
    container = soup.find('tbody')
    sixth_col = []
    count = 5
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            sixth_col.append(val)
        
        count += 7    
    return sixth_col

In [29]:
def get_7seventh(soup):
    container = soup.find('tbody')
    seventh_col = []
    count = 6
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            seventh_col.append(val)
        
        count += 7    
    return seventh_col

### 14 Stats

In [217]:
def get_14first(soup):
    container = soup.find('tbody')
    first_col = []
    count = 0
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            first_col.append(val)
        
        count += 14    
    return first_col

In [218]:
def get_14second(soup):
    container = soup.find('tbody')
    second_col = []
    count = 1
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            second_col.append(val)
        
        count += 14    
    return second_col

In [219]:
def get_14third(soup):
    container = soup.find('tbody')
    third_col = []
    count = 2
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            third_col.append(val)
        
        count += 14    
    return third_col

In [220]:
def get_14fourth(soup):
    container = soup.find('tbody')
    fourth_col = []
    count = 3
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            fourth_col.append(val)
        
        count += 14    
    return fourth_col

In [221]:
def get_14fifth(soup):
    container = soup.find('tbody')
    fifth_col = []
    count = 4
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            fifth_col.append(val)
        
        count += 14    
    return fifth_col

In [222]:
def get_14sixth(soup):
    container = soup.find('tbody')
    sixth_col = []
    count = 5
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            sixth_col.append(val)
        
        count += 14
    return sixth_col

In [223]:
def get_14seventh(soup):
    container = soup.find('tbody')
    seventh_col = []
    count = 6
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            seventh_col.append(val)
        
        count += 14    
    return seventh_col

In [224]:
def get_14eighth(soup):
    container = soup.find('tbody')
    eighth_col = []
    count = 7
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            eighth_col.append(val)
        
        count += 14    
    return eighth_col

In [225]:
def get_14ninth(soup):
    container = soup.find('tbody')
    ninth_col = []
    count = 8
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            ninth_col.append(val)
        
        count += 14    
    return ninth_col

In [226]:
def get_14tenth(soup):
    container = soup.find('tbody')
    tenth_col = []
    count = 9
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            tenth_col.append(val)
        
        count += 14    
    return tenth_col

In [227]:
def get_14eleventh(soup):
    container = soup.find('tbody')
    eleventh_col = []
    count = 10
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            eleventh_col.append(val)
        
        count += 14    
    return eleventh_col

In [228]:
def get_14twelfth(soup):
    container = soup.find('tbody')
    twelfth_col = []
    count = 11
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            twelfth_col.append(val)
        
        count += 14    
    return twelfth_col

In [229]:
def get_14thirteenth(soup):
    container = soup.find('tbody')
    thirteenth_col = []
    count = 12
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            thirteenth_col.append(val)
        
        count += 14    
    return thirteenth_col

In [231]:
def get_14fourteenth(soup):
    container = soup.find('tbody')
    fourteenth_col = []
    count = 13
    length = len(container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number'))
    for i in range(length):
        if count in range(length):
            val = container.findAll('td', class_ = 'TableBase-bodyTd TableBase-bodyTd--number')[count].get_text()
            val = val.replace('\n', '')
            val = val.replace(' ', '')
            fourteenth_col.append(val)
        
        count += 14    
    return fourteenth_col

### DataFrame Functions

In [234]:
def get_passing(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    
    passing_df['PLAYER'] = get_name(soup)
    passing_df['POSITION'] = get_position(soup)
    passing_df['TEAM'] = get_team(soup)
    passing_df['GP'] = get_12first(soup)
    passing_df['ATT'] = get_12second(soup)
    passing_df['CMP'] = get_12third(soup)
    passing_df['PCT'] = get_12fourth(soup)
    passing_df['YDS'] = get_12fifth(soup)
    passing_df['YDSpG'] = get_12sixth(soup)
    passing_df['LNG'] = get_12seventh(soup)
    passing_df['TD'] = get_12eighth(soup)
    passing_df['INT'] = get_12ninth(soup)
    passing_df['SCK'] = get_12tenth(soup)
    passing_df['YDSL'] = get_12eleventh(soup)
    passing_df['RATE'] = get_12twelfth(soup)    

In [214]:
def get_kicking(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    
    kicking_df['PLAYER'], kicking_df['POSITION'], kicking_df['TEAM'] = get_name_pos_team_missing_team(soup)
    kicking_df['GP'] = get_12first(soup)
    kicking_df['FGM_A'] = get_12second(soup)
    kicking_df['FG%'] = get_12third(soup)
    kicking_df['LNG'] = get_12fourth(soup)
    kicking_df['_1_19'] = get_12fifth(soup)
    kicking_df['_20_29'] = get_12sixth(soup)
    kicking_df['_30_39'] = get_12seventh(soup)
    kicking_df['_40_49'] = get_12eighth(soup)
    kicking_df['OVER50'] = get_12ninth(soup)
    kicking_df['XPM_A'] = get_12tenth(soup)
    kicking_df['XP%'] = get_12eleventh(soup)
    kicking_df['PTS'] = get_12twelfth(soup)    

In [241]:
def get_rushing(url_list):
    name_list = []
    position_list = []
    team_list = []
    gp_list = []
    att_list = []
    yds_list = []
    ydspg_list = []
    avg_list = []
    td_list = []
    lng_list = []
    
    names = []
    positions = []
    teams = []
    
    for url in url_list:
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser')
        
        names, positions, teams = get_name_pos_team_missing_team(soup)
    
        name_list += names
        position_list += positions
        team_list += teams
        gp_list += get_7first(soup)
        att_list += get_7second(soup)
        yds_list += get_7third(soup)
        ydspg_list += get_7fourth(soup)
        avg_list += get_7fifth(soup)
        td_list += get_7sixth(soup)
        lng_list += get_7seventh(soup)
    
    rushing_df['PLAYER'] = name_list
    rushing_df['POSITION'] = position_list
    rushing_df['TEAM'] = team_list
    rushing_df['GP'] = gp_list
    rushing_df['ATT'] = att_list
    rushing_df['YDS'] = yds_list
    rushing_df['YDSpG'] = ydspg_list
    rushing_df['AVG'] = avg_list
    rushing_df['TD'] = td_list
    rushing_df['LNG'] = lng_list

In [43]:
def get_receiving(url_list):
    name_list = []
    position_list = []
    team_list = []
    gp_list = []
    rec_list = []
    yds_list = []
    ydspg_list = []
    avg_list = []
    lng_list = []
    td_list = []
    
    for url in url_list:
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser')
    
        name_list += get_name(soup)
        position_list += get_position(soup)
        team_list += get_team(soup)
        gp_list += get_7first(soup)
        rec_list += get_7second(soup)
        yds_list += get_7third(soup)
        ydspg_list += get_7fourth(soup)
        avg_list += get_7fifth(soup)
        lng_list += get_7sixth(soup)
        td_list += get_7seventh(soup)
    
    receiving_df['PLAYER'] = name_list
    receiving_df['POSITION'] = position_list
    receiving_df['TEAM'] = team_list
    receiving_df['GP'] = gp_list
    receiving_df['REC'] = att_list
    receiving_df['YDS'] = yds_list
    receiving_df['YDSpG'] = ydspg_list
    receiving_df['AVG'] = avg_list
    receiving_df['LNG'] = lng_list
    receiving_df['TD'] = td_list

In [232]:
def get_defense(url_list):
    name_list = []
    position_list = []
    team_list = []
    gp_list = []
    stk_list = []
    atk_list = []
    cmb_list = []
    int_list = []
    yds_list = []
    lng_list = []
    td_list = []
    fum_list = []
    frec_list = []
    ftd_list = []
    sck_list = []    
    pdef_list = []    
    sfty_list = []
    
    
    for url in url_list:
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser')
    
        name_list += get_name(soup)
        position_list += get_position(soup)
        team_list += get_team(soup)
        gp_list += get_14first(soup)
        stk_list += get_14second(soup)
        atk_list += get_14third(soup)
        cmb_list += get_14fourth(soup)
        int_list += get_14fifth(soup)
        yds_list += get_14sixth(soup)
        lng_list += get_14seventh(soup)
        td_list += get_14eighth(soup)
        fum_list += get_14ninth(soup)
        frec_list += get_14tenth(soup)
        ftd_list += get_14eleventh(soup)
        sck_list += get_14twelfth(soup)
        pdef_list += get_14thirteenth(soup)
        sfty_list += get_14fourteenth(soup)
    
    defense_df['PLAYER'] = name_list
    defense_df['POSITION'] = position_list
    defense_df['TEAM'] = team_list
    defense_df['GP'] = gp_list
    defense_df['STK'] = stk_list
    defense_df['ATK'] = atk_list
    defense_df['CMB'] = cmb_list
    defense_df['INT'] = int_list
    defense_df['YDS'] = yds_list
    defense_df['LNG'] = lng_list
    defense_df['TD'] = td_list
    defense_df['FMB'] = fum_list
    defense_df['FREC'] = frec_list
    defense_df['FTD'] = ftd_list
    defense_df['SCK'] = sck_list
    defense_df['PDEF'] =pdef_list
    defense_df['SFTY'] = sfty_list

### Scraping Sites

#### Passing

In [235]:
url = 'https://www.cbssports.com/nfl/stats/player/passing/nfl/regular/all/'

get_passing(url)

In [236]:
passing_df

,PLAYER,POSITION,TEAM,GP,ATT,CMP,PCT,YDS,YDSpG,LNG,TD,INT,SCK,YDSL,RATE
0,Matt Ryan,QB,ATL,1,54,37,68.5,450,450.0,44,2,1,2,16,98.5
1,Aaron Rodgers,QB,GB,1,44,32,72.7,364,364.0,45,4,0,0,0,127.5
2,Philip Rivers,QB,IND,1,46,36,78.3,363,363.0,35,1,2,0,0,88.7
3,Russell Wilson,QB,SEA,1,35,31,88.6,322,322.0,38,4,0,3,23,143.1
4,Josh Allen,QB,BUF,1,46,33,71.7,312,312.0,29,2,0,3,6,104.6
5,Matthew Stafford,QB,DET,1,42,24,57.1,297,297.0,32,1,1,1,9,77.2
6,Daniel Jones,QB,NYG,1,41,26,63.4,279,279.0,41,2,2,3,17,79.2
7,Jared Goff,QB,LAR,1,31,20,64.5,275,275.0,31,0,1,1,6,79.4
8,Lamar Jackson,QB,BAL,1,25,20,80.0,275,275.0,47,3,0,2,14,152.1
9,Teddy Bridgewater,QB,CAR,1,34,22,64.7,270,270.0,75,1,0,1,10,98.9


#### Rushing

In [239]:
url_1 = 'https://www.cbssports.com/nfl/stats/player/rushing/nfl/regular/all/'
url_list = [url_1]

no_pages = 3

for i in range(2, no_pages + 1):
    url = 'https://www.cbssports.com/nfl/stats/player/rushing/nfl/regular/all/?page=' + str(i)
    url_list.append(url)

url_list

['https://www.cbssports.com/nfl/stats/player/rushing/nfl/regular/all/',
 'https://www.cbssports.com/nfl/stats/player/rushing/nfl/regular/all/?page=2',
 'https://www.cbssports.com/nfl/stats/player/rushing/nfl/regular/all/?page=3']

In [242]:
get_rushing(url_list)

ValueError: Length of values does not match length of index